In [2]:
#Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import re 
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',lambda x: '%.3f' % x)
df_ = pd.read_csv("flo_data_20k.csv")
df = df_.copy()

In [3]:
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

In [4]:
#Check first 5 Customer information and values of the columns. 
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR]


In [6]:
#Writing functions to suppress outliers.
def outlier_threshold(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3-quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_threshold(dataframe,variable):
    low_limit, up_limit = outlier_threshold(dataframe,variable)
    dataframe.loc[(dataframe[variable] < low_limit),variable] = low_limit.round()
    dataframe.loc[(dataframe[variable] > up_limit),variable] = up_limit.round()


In [16]:
#Applying the supression process.
replace_with_threshold(df,"order_num_total_ever_online")
replace_with_threshold(df,"order_num_total_ever_offline")
replace_with_threshold(df,"customer_value_total_ever_offline")
replace_with_threshold(df,"customer_value_total_ever_online")

In [8]:
#Total for each customer number of purchases and spending.
#Total total_transaction = order_num_total_ever_online + order_num_total_ever_offline
#Total Price for each customer: customer_value_total_ever_online + customer_value_total_ever_offline
df["total_price"] =  df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
df["total_transaction"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]

In [9]:
#Converting String types to date type.
df["first_order_date"] = pd.to_datetime(df["first_order_date"])
df["last_order_date"] = pd.to_datetime(df["last_order_date"])
df["last_order_date_online"] = pd.to_datetime(df["last_order_date_online"])
df["last_order_date_offline"] = pd.to_datetime(df["last_order_date_offline"])

In [10]:
#Analyse Date
today_date = dt.datetime(2021,6,1)

In [13]:
#Nomination of recency_cltv_weekly variable.
df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]) / dt.timedelta(weeks = 1)

In [17]:
#Nomination of T_weekly variable.
df["T_weekly"] = (today_date - df["first_order_date"]) / dt.timedelta(weeks = 1)

In [18]:
df["frequency"] = df["total_transaction"]

In [19]:
#In this dataset, we select customers who have interacted with the company more than once. In order to explain the recency value on a customer basis, it is necessary to look at the date between the last and first purchase, so we select the ones with more than 1 frequency.
df = df[(df["frequency"] > 1)]

In [20]:
df["monetary_cltv_avg"] = df["total_price"] / df["frequency"]

In [21]:
cltv_df = df.groupby("master_id").agg({"recency_cltv_weekly": lambda x: x,"T_weekly": lambda y: y,"frequency":lambda a: a,"monetary_cltv_avg":lambda z:z})

In [22]:
cltv_df

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
master_id,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571,80.000,5.000,155.214
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571,290.143,3.000,89.823
000be838-85df-11ea-a90b-000d3a38a36f,28.286,58.714,4.000,180.673
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.429,51.286,7.000,124.880
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.286,46.143,7.000,231.476
...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,85.429,111.571,5.000,159.374
fff4736a-60a4-11ea-8dd8-000d3a38a36f,46.714,56.286,2.000,159.435
fffacd34-ae14-11e9-a2fc-000d3a38a36f,100.571,137.286,6.000,163.987


In [24]:
bgf = BetaGeoFitter(penalizer_coef = 0.001)

In [25]:
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

C:\Users\scuto\anaconda3\lib\site-packages\pandas\core\arraylike.py:405: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [29]:
#Expectation sales for next 3 months.
cltv_df["exp_sales_3_month"] = bgf.predict(12,cltv_df['frequency'],
                                    cltv_df['recency_cltv_weekly'],
                                    cltv_df['T_weekly'])

In [30]:
#Expectation sales for next 6 months
cltv_df["exp_sales_6_month"] = bgf.predict(24,cltv_df['frequency'],
                                    cltv_df['recency_cltv_weekly'],
                                    cltv_df['T_weekly'])

In [31]:
cltv_df

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
master_id,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571,80.000,5.000,155.214,0.666,1.331
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571,290.143,3.000,89.823,0.218,0.437
000be838-85df-11ea-a90b-000d3a38a36f,28.286,58.714,4.000,180.673,0.682,1.364
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.429,51.286,7.000,124.880,1.004,2.008
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.286,46.143,7.000,231.476,1.046,2.092
...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,85.429,111.571,5.000,159.374,0.554,1.107
fff4736a-60a4-11ea-8dd8-000d3a38a36f,46.714,56.286,2.000,159.435,0.513,1.026
fffacd34-ae14-11e9-a2fc-000d3a38a36f,100.571,137.286,6.000,163.987,0.543,1.086


In [32]:
ggf = GammaGammaFitter(penalizer_coef = 0.01)

ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [33]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time= 6, 
                                   freq="W",  
                                   discount_rate=0.01)

In [34]:
cltv.reset_index()

,master_id,clv
0,00016786-2f5a-11ea-bb80-000d3a38a36f,223.666
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,43.623
2,000be838-85df-11ea-a90b-000d3a38a36f,268.504
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,269.223
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,518.899
...,...,...
19940,fff1db94-afd9-11ea-b736-000d3a38a36f,191.014
19941,fff4736a-60a4-11ea-8dd8-000d3a38a36f,185.703
19942,fffacd34-ae14-11e9-a2fc-000d3a38a36f,191.796
19943,fffacecc-ddc3-11e9-a848-000d3a38a36f,298.071


In [35]:
cltv_final = cltv_df.merge(cltv, on = "master_id", how = "left")

In [36]:
cltv_final

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,clv
master_id,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571,80.000,5.000,155.214,0.666,1.331,223.666
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571,290.143,3.000,89.823,0.218,0.437,43.623
000be838-85df-11ea-a90b-000d3a38a36f,28.286,58.714,4.000,180.673,0.682,1.364,268.504
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.429,51.286,7.000,124.880,1.004,2.008,269.223
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.286,46.143,7.000,231.476,1.046,2.092,518.899
...,...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,85.429,111.571,5.000,159.374,0.554,1.107,191.014
fff4736a-60a4-11ea-8dd8-000d3a38a36f,46.714,56.286,2.000,159.435,0.513,1.026,185.703
fffacd34-ae14-11e9-a2fc-000d3a38a36f,100.571,137.286,6.000,163.987,0.543,1.086,191.796


In [37]:
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],cltv_df['monetary_cltv_avg'])

In [38]:
#Get final cltv for each customers.
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time= 6, 
                                   freq="W",  
                                   discount_rate=0.01)

In [39]:
cltv.reset_index()

,master_id,clv
0,00016786-2f5a-11ea-bb80-000d3a38a36f,223.666
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,43.623
2,000be838-85df-11ea-a90b-000d3a38a36f,268.504
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,269.223
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,518.899
...,...,...
19940,fff1db94-afd9-11ea-b736-000d3a38a36f,191.014
19941,fff4736a-60a4-11ea-8dd8-000d3a38a36f,185.703
19942,fffacd34-ae14-11e9-a2fc-000d3a38a36f,191.796
19943,fffacecc-ddc3-11e9-a848-000d3a38a36f,298.071


In [67]:
cltv_final = cltv_df.merge(cltv, on = "master_id", how = "left")

In [68]:
cltv_final

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,expected_average_profit,clv
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571,80.000,5.000,395.866,0.666,1.331,411.406,574.689
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571,290.143,3.000,139.983,0.218,0.437,150.216,68.807
000be838-85df-11ea-a90b-000d3a38a36f,28.286,58.714,4.000,359.158,0.682,1.364,377.036,539.403
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.429,51.286,7.000,450.524,1.004,2.008,462.949,975.304
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.286,46.143,7.000,1028.816,1.046,2.092,1056.500,2319.332
...,...,...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,85.429,111.571,5.000,469.544,0.554,1.107,487.835,566.857
fff4736a-60a4-11ea-8dd8-000d3a38a36f,46.714,56.286,2.000,159.435,0.513,1.026,177.213,190.797
fffacd34-ae14-11e9-a2fc-000d3a38a36f,100.571,137.286,6.000,779.947,0.543,1.086,804.692,917.330


In [40]:
cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels = ["D","C","B","A"])

In [41]:
cltv_final

,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,clv,segment
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,78.571,80.000,5.000,155.214,0.666,1.331,223.666,B
00034aaa-a838-11e9-a2fc-000d3a38a36f,247.571,290.143,3.000,89.823,0.218,0.437,43.623,D
000be838-85df-11ea-a90b-000d3a38a36f,28.286,58.714,4.000,180.673,0.682,1.364,268.504,A
000c1fe2-a8b7-11ea-8479-000d3a38a36f,47.429,51.286,7.000,124.880,1.004,2.008,269.223,A
000f5e3e-9dde-11ea-80cd-000d3a38a36f,43.286,46.143,7.000,231.476,1.046,2.092,518.899,A
...,...,...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,85.429,111.571,5.000,159.374,0.554,1.107,191.014,B
fff4736a-60a4-11ea-8dd8-000d3a38a36f,46.714,56.286,2.000,159.435,0.513,1.026,185.703,B
fffacd34-ae14-11e9-a2fc-000d3a38a36f,100.571,137.286,6.000,163.987,0.543,1.086,191.796,B


In [45]:
#TR
#Yönetime Öneri: 
#Tavsiye verilecek segmentler A ve D 

# A grubu segmentin CLV değerleri oldukça yüksektir. Müşterilere satın alma alışkanlıkları yöneliminde kampanya ve indirimler sağlanmalı, böylece toplam 
#getiri arttırılma yoluna gidilmeli. 

#D Segment grubu, CLV değerlerinin en düşük olduğu grup. Satın alma alışkanlıkları üzerinde etkili olan faktörler incelenip, müşterinin satın alım 
#frekansını arttırmaya ve birim bazda pahalı ürünler almaya yönlendirecek kampanya ve indirimler sağlanmalı.

#ING
# Suggestion to the Management:
#Recommended segments A and D

# CLV values of group A segment are quite high. Customers should be provided with campaigns and discounts in the direction of their purchasing habits, so that the total
#return should be increased.

#D Segment group is the group with the lowest CLV values. The factors affecting the purchasing habits are examined and the customer's purchasing habits are analyzed.
#Campaigns and discounts should be provided to increase frequency and to buy expensive products on a unit basis.

In [43]:
cltv_final.groupby('segment').agg(['max', 'mean', 'count']).T

segment                          D        C        B        A
recency_cltv_weekly max    432.571  433.000  415.571  433.429
                    mean   139.000   92.630   81.988   67.427
                    count 4987.000 4986.000 4986.000 4986.000
T_weekly            max    436.286  434.714  423.571  437.143
                    mean   162.183  112.818  100.327   82.550
                    count 4987.000 4986.000 4986.000 4986.000
frequency           max     18.000   32.000   50.000   57.000
                    mean     3.769    4.405    5.093    6.647
                    count 4987.000 4986.000 4986.000 4986.000
monetary_cltv_avg   max    284.985  342.840  539.985 1401.800
                    mean    93.152  125.789  160.637  228.831
                    count 4987.000 4986.000 4986.000 4986.000
exp_sales_3_month   max      1.050    1.568    1.782    4.656
                    mean     0.409    0.525    0.601    0.773
                    count 4987.000 4986.000 4986.000 4986.000
exp_sales_6_month   max      2.100    3.137    3.564    9.312
                    mean     0.818    1.051    1.202    1.546
                    count 4987.000 4986.000 4986.000 4986.000
clv                 max    112.245  165.469  240.058 3327.777
                    mean    80.340  138.312  199.533  362.316
                    count 4987.000 4986.000 4986.000 4986.000